In [1]:
OPENAI_API_KEY = ""

from openai import OpenAI 
import os
import base64

MODEL="gpt-4o-mini"
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", OPENAI_API_KEY))

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

def extract_text(base64_image):
    response = client.chat.completions.create(
        model = MODEL,
        messages=[
            {"role": "system", "content": "You are a helpful assistant that can extract text from an image and return in a latex markdown format"},
            {"role": "user", "content": [
                {"type": "text", "text": "Can you extract all the text from this image?"},
                {"type": "image_url", "image_url": {
                    "url": f"data:image/png;base64,{base64_image}"}
                }
            ]}
        ],
        temperature=0.0,
    )

    return response.choices[0].message.content

def save_to_markdown(text, path):
    with open(path, "w", encoding="utf-8") as f:
        f.write(text)
    
    print(f"saved to {path}")

In [3]:
from natsort import natsorted
from tqdm.auto import tqdm
import os

text = ''
FOLDER_PATH = r"E:\pi_school\eva-work-notes\text-extraction\data\s3\ground_truths\elsevier\two"
for image in natsorted(os.listdir(FOLDER_PATH)):
    base64_image = encode_image(os.path.join(FOLDER_PATH, image))
    text += extract_text(base64_image)

save_to_markdown(text, FOLDER_PATH + '.md')

saved to E:\pi_school\eva-work-notes\text-extraction\data\s3\ground_truths\elsevier\two.md


In [9]:
GEMINI_KEY = ''

import google.generativeai as genai

genai.configure(api_key=GEMINI_KEY)

model = genai.GenerativeModel('gemini-2.0-flash')
# response = model.generate_content("The opposite of hot is")
# print(response.text)

from PIL import Image
from IPython.display import display
import base64
import io
import os
from tqdm.auto import tqdm

FOLDER = "../data/s3/ground_truths/elsevier/two/"

def save_to_markdown(text, path):
    with open(path, "w", encoding="utf-8") as f:
        f.write(text)
    
    print(f"saved to {path}")

def ocr_with_gemini(image_paths):
    """Process images with Gemini 2.0 Flash for OCR"""
    images = [Image.open(os.path.join(FOLDER, path)) for path in tqdm(os.listdir(image_paths))]
    
    prompt = """
    Extract all the text, tables and formulas in a latex markdown format.
    """
    
    response = model.generate_content([prompt, images[6]])
    return response.text

text = ocr_with_gemini(FOLDER)
save_to_markdown(text, FOLDER + '_gemini6.md')


100%|██████████| 28/28 [00:00<00:00, 4681.14it/s]


saved to ../data/s3/ground_truths/elsevier/two/_gemini6.md


In [4]:
# OPENAI_API_KEY = ""

# from openai import OpenAI 
# import os
# import base64

# MODEL="gpt-4o-mini"
# client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", OPENAI_API_KEY))

# def encode_image(image_path):
#     with open(image_path, "rb") as image_file:
#         return base64.b64encode(image_file.read()).decode("utf-8")

# def extract_text(base64_image):
#     response = client.chat.completions.create(
#         model = MODEL,
#         messages=[
#             {"role": "system", "content": "You are a helpful assistant that can extract text from an image and return in a latex markdown format"},
#             {"role": "user", "content": [
#                 {"type": "text", "text": "Can you extract all the text from this image?"},
#                 {"type": "image_url", "image_url": {
#                     "url": f"data:image/png;base64,{base64_image}"}
#                 }
#             ]}
#         ],
#         temperature=0.0,
#     )

#     return response.choices[0].message.content

# def save_to_markdown(text, path):
#     with open(path, "w", encoding="utf-8") as f:
#         f.write(text)
    
#     print(f"saved to {path}")

# from natsort import natsorted
# from tqdm.auto import tqdm
# import os

# text = ''
# base64_image = encode_image(r"E:\pi_school\eva-work-notes\text-extraction\data\s3\ground_truths\esa\06d48fa6-6c6b-41af-bc07-6f7cabc080d1_page-0001.jpg")
# text += extract_text(base64_image)

# save_to_markdown(text, '../data/s3/ground_truths/esa/06d48fa6-6c6b-41af-bc07-6f7cabc080d1' + '.md')

In [11]:
# #### test extraction format

# file = r"E:\pi_school\eva-work-notes\text-extraction\data\s3\original\iop\02290c1a-d45a-4bfc-b1e2-819da8350b56.pdf"

# from markitdown import MarkItDown

# md = MarkItDown()
# result = md.convert(file)

In [8]:
# print(result.text_content)

In [10]:
# !nougat pdf ../data/s3/original/iop/02290c1a-d45a-4bfc-b1e2-819da8350b56.pdf -o output

In [ ]:
### try qwen --> tried on colab (not great)

# from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
# from PIL import Image

# # Load the model and processor
# model = Qwen2VLForConditionalGeneration.from_pretrained("Qwen/Qwen2-VL-2B-Instruct", torch_dtype="auto", device_map="auto")
# processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct")

In [3]:

# FOLDER = "../data/s3/ground_truths/elsevier/two/"

# images = [Image.open(os.path.join(FOLDER, path)) for path in tqdm(os.listdir(FOLDER))]

In [6]:
### marker ocr